<h1>Predicting game prices</h1>
<h5 style="margin-left: 2rem">By: Shalev Hadar, Elad Ben-Haim</h5>
<p>Gmail/Email user: shaleveladhit@gmail.com</p>
<p>Steam Username: ShalevEladHit</p>
<p>Steam Password: ShalevElad123@</p>
<br/>
<br/>

<h4>Defining our problem</h4>
<table dir="rtl">
    <tr>
        <th>
            מה הבעיה
        </th>
        <th>
            תיאור הבעיה
        </th>
    </tr>
    <tr>
        <td>
        האם ניתן לחזות מחיר של משחק בעוד x זמן?
        </td>
        <td>
        מתי הכי ישתלם לקנות את המשחק בעתיד
        </td>
    </tr>
    <tr>
        <td>
        האם ניתן לחזור מתי למוכר הכי משתלם לעשות מבצע על המשחק?
        </td>
        <td>
        מתי הכי כדאי למוכר לעשות מבצע כדי להביא עוד שחקנים ולהמשיך למכור עם הרווח הגדול ביותר
        </td>
    </tr>
</table>
<br/>


<br/>

<h4>Importing</h6>

In [49]:
import requests
import bs4
from bs4 import BeautifulSoup
import random
import itertools

<h4>Global functions</h4>

In [62]:
def get_html_response(url: str) -> requests.Response:
    return requests.get(url)

def get_response_as_beautiful_soup(htmlText: requests.Response) -> BeautifulSoup:
    return BeautifulSoup(htmlText.text, 'html.parser')

<br/>

<h4>Defining proxies for scraping</h4>

<h5>Get proxy list response html website</h5>
<h6>get the html as response object instead of getting the html again and again</h6>

In [55]:
# Get the html of the proxy list website
def get_proxy_list_html() -> requests.Response:
    # Website to get free proxies
    return get_html_response('https://free-proxy-list.net/')

proxies_response = get_proxy_list_html()

<h5>Scrape proxy ip addresses</h5>
<h6>gets the ip addresses as a list, shuffles them and returns an iterator to cycle through when making scrape requests</h6>

In [63]:
def get_proxy_list() -> itertools.cycle:
    soup = get_response_as_beautiful_soup(proxies_response)
    proxy_soup_list = soup.select('#list > div > div.table-responsive > div > table > tbody > tr')
    proxy_list = list(map(lambda i: i.select('td:nth-child(1)')[0].text + ':' + i.select('td:nth-child(2)')[0].text, proxy_soup_list))
    random.shuffle(proxy_list)
    return itertools.cycle(proxy_list)


<br/>

<h4>Scrape isThereAnyDeal website</h4>
<h6>Steps:</h6>
<ol>
<li>Crawl list of top 100 trending games</li>
<li>For each game, get the game details from steam API</li>
<li>For each game, crawl price history on isThereAnyDeal</li>
</ol>

<h5>Get list of 100 top trending games</h5>

In [64]:
def get_is_there_any_deal_games_response() -> requests.Response:
    filteredUrl = 'https://isthereanydeal.com/?by=trending:desc#/filter:&releaseyear/2010/2021,&pl/windows,&drm/steam,steam,-dlc,-type/6,-type/8,-type/7;/options:all'
    return get_html_response(filteredUrl)
is_there_any_deal_games_response = get_is_there_any_deal_games_response()

In [ ]:
def get_game_links():
    soup = get_response_as_beautiful_soup(is_there_any_deal_games_response)
    

<h3>Used Resources</h3>
<dl>
    <dt>Scraping</dt>
    <dd><a href="https://isthereanydeal.com/game/reddeadredemptionii/info/">Is-There-Any-Deal website</a></dd>
    <dd><a href="https://www.geeksforgeeks.org/web-scraping-without-getting-blocked/">Using Proxies to avoiding getting blocked</a></dd>
</dl>